# Diheadral angles of conformers of Ethane

<div align="right"> by Qkrishi</div>



In [2]:
!date

Tue Dec 20 15:01:19 IST 2022


## Defining the problem in PySCF driver

In [7]:
from qiskit_nature.units import DistanceUnit
from qiskit_nature.second_q.drivers import PySCFDriver

driver = PySCFDriver(
    
    
    atom="C 0.00000000 0.00000000 0.76532900; H -0.51007400 0.88446900 1.16448900; H 1.02100900   -0.00049700    1.16448900; H -0.51093500   -0.88397100    1.16448900;  C 0.00000000 0.00000000 -0.76532900; H 0.51093500 -0.88397100 -1.164489; H -1.02100900 -0.00049700 -1.16448900; H 0.51007400 0.88446900 -1.16448900",
       
    
    basis="sto3g",
    charge=0,
    spin=0,
    unit=DistanceUnit.ANGSTROM
)

es_problem = driver.run()

In [8]:
problem = driver.run()
print(problem)

## The Jordan-Wigner Mapping


In [9]:
%%time
from qiskit_nature.second_q.mappers import JordanWignerMapper
mapper = JordanWignerMapper()
fermionic_op = problem.hamiltonian.second_q_op()
qubit_jw_op = mapper.map(fermionic_op)
print(qubit_jw_op.num_qubits)

32
CPU times: user 1min 11s, sys: 1.03 s, total: 1min 12s
Wall time: 1min 12s


### Qubit Conversion

In [14]:
%%time
from qiskit_nature.second_q.mappers import QubitConverter

converter = QubitConverter(JordanWignerMapper(), z2symmetry_reduction="auto")

qubit_op = converter.convert(fermionic_op, sector_locator=problem.symmetry_sector_locator)
print(qubit_op.num_qubits)

29
CPU times: user 1min 23s, sys: 592 ms, total: 1min 23s
Wall time: 1min 23s


## Pairity Mapping

In [ ]:
%%time

from qiskit_nature.second_q.mappers import ParityMapper

mapper = ParityMapper()

qubit_p_op = mapper.map(fermionic_op)
print(qubit_p_op.num_qubits)

32
CPU times: user 1min 8s, sys: 512 ms, total: 1min 8s
Wall time: 1min 8s


In [53]:
%%time
converter = QubitConverter(ParityMapper(), two_qubit_reduction=True)
qubit_op = converter.convert(fermionic_op, num_particles=problem.num_particles)
print(qubit_op.num_qubits)

30
CPU times: user 1min 15s, sys: 376 ms, total: 1min 15s
Wall time: 1min 15s


In [52]:
%%time

converter = QubitConverter(ParityMapper(), two_qubit_reduction=True) #, z2symmetry_reduction="auto")

qubit_op = converter.convert(fermionic_op, sector_locator=problem.symmetry_sector_locator)
print(qubit_op.num_qubits)

32
CPU times: user 1min 7s, sys: 352 ms, total: 1min 8s
Wall time: 1min 8s


## Initialise VQE

In [54]:
%%time

from qiskit.algorithms.optimizers import SLSQP
from qiskit.primitives import Estimator
from qiskit_nature.second_q.algorithms import VQEUCCFactory
from qiskit_nature.second_q.circuit.library import UCCSD

vqe_solver = VQEUCCFactory(Estimator(), UCCSD(), SLSQP())


from qiskit.algorithms.minimum_eigensolvers import VQE
from qiskit.circuit.library import TwoLocal

tl_circuit = TwoLocal(
    rotation_blocks=["h", "rx"],
    entanglement_blocks="cz",
    entanglement="full",
    reps=2,
    parameter_prefix="y",
)

from qiskit_nature.second_q.algorithms import GroundStateEigensolver

calc = GroundStateEigensolver(converter, vqe_solver)

CPU times: user 5.59 ms, sys: 13 µs, total: 5.6 ms
Wall time: 5.39 ms
